# Contextual based neural lemmatization with Flair

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install --upgrade flair

Requirement already up-to-date: flair in /usr/local/lib/python3.6/dist-packages (0.7)


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings, CharacterEmbeddings
from typing import List

# ASSIGNMENT 1

Please inspect the data we provided (train, dev and test sets available): 'content/drive/My Drive/Colab Notebooks/2022-ILTAPP/datasets/sigmorphon2019/'

+ TODO Try to spot the differences with the Lab 04-ner-training-tagging_TODO.ipynb.
+ TODO: define the columns required to instantiate the ColumnCorpus and the tag we want to predict.
+ TODO: use downsampling to speed up training.
+ TODO: print the total number of tags to predict in the corpus and check how this changes as you change the downsampling.
+ TODO: experiment with Classic Embeddings, CharacterEmbeddings and Flair Embeddings for improving accuracy performance (also with number of epochs).
  + HINT: Do not combine more than two types of embeddings to avoid running out RAM.

In [ ]:
# TODO define columns

# TODO get the corpus


2021-02-17 10:11:35,640 Reading data from /content/drive/My Drive/datasets/sigmorphon2019
2021-02-17 10:11:35,641 Train: /content/drive/My Drive/datasets/sigmorphon2019/ud-ses-true-train.tsv
2021-02-17 10:11:35,641 Dev: /content/drive/My Drive/datasets/sigmorphon2019/ud-ses-true-dev.tsv
2021-02-17 10:11:35,642 Test: /content/drive/My Drive/datasets/sigmorphon2019/ud-ses-true-test.tsv


In [ ]:
# TODO what tag do we want to predict?

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# TODO print total number of tags


[b'<unk>', b'O', b'\xe2\x86\x930;d\xc2\xa6+', b'\xe2\x86\x930;d\xc2\xa6--+', b'\xe2\x86\x930;d\xc2\xa6----+', b'\xe2\x86\x930;d--+e\xc2\xa6+n+', b'\xe2\x86\x930;d\xc2\xa6-+', b'\xe2\x86\x930;d\xc2\xa6---+n+', b'\xe2\x86\x930;d\xc2\xa6----\xe2\x86\x92-+u+', b'\xe2\x86\x930;d-\xc2\xa6--+k+a+n+', b'\xe2\x86\x910\xc2\xa6\xe2\x86\x931;d\xc2\xa6-+', b'\xe2\x86\x930;d\xc2\xa6----+u+', b'\xe2\x86\x930;d+*+e\xc2\xa6+n+', b'\xe2\x86\x910\xc2\xa6\xe2\x86\x931;d\xc2\xa6--+', b'\xe2\x86\x930;d-\xc2\xa6+k+a+n+', b'\xe2\x86\x910\xc2\xa6\xe2\x86\x933\xc2\xa6\xe2\x86\x91-3\xc2\xa6\xe2\x86\x93-1;d\xc2\xa6-+', b'\xe2\x86\x930;d+*+e\xc2\xa6-\xe2\x86\x92+', b'\xe2\x86\x930;d\xc2\xa6---+u+', b'\xe2\x86\x910\xc2\xa6\xe2\x86\x931;d\xc2\xa6---+', b'\xe2\x86\x930;d\xc2\xa6----+a+u+', b'\xe2\x86\x930;d\xc2\xa6---+', b'\xe2\x86\x930;d\xc2\xa6----+i+', b'\xe2\x86\x930;d\xc2\xa6---+i+', b'\xe2\x86\x930;d-\xc2\xa6--+z+a\xe2\x86\x92+', b'\xe2\x86\x930;d\xc2\xa6++\xe2\x86\x92++\xe2\x86\x92\xe2\x86\x92\xe2\x86\x92\xe2\

In [ ]:
# TODO play with embeddings and check performance; which one is the best combination?
embedding_types: List[TokenEmbeddings] = [

    CharacterEmbeddings(),
    #WordEmbeddings('eu'),
    FlairEmbeddings('eu-forward', chars_per_chunk=128),
    FlairEmbeddings('eu-backward', chars_per_chunk=128),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)


In [ ]:
# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type
)

In [ ]:
# initialize trainer
from flair.trainers import ModelTrainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.train('/content/drive/My Drive/Colab Notebooks/2022-ILTAPP/eu-lemma-model', mini_batch_size=16, train_with_dev=False, max_epochs=20)

2021-02-17 10:11:39,929 ----------------------------------------------------------------------------------------------------
2021-02-17 10:11:39,932 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4488, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4488, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(4488, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=4488, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): 

{'dev_loss_history': [23.650285720825195,
  17.585355758666992,
  15.389253616333008,
  12.943793296813965,
  11.968303680419922,
  11.547333717346191,
  9.920639038085938,
  9.87789249420166,
  9.183420181274414,
  8.54510498046875,
  8.409395217895508,
  8.142658233642578,
  7.792398929595947,
  7.770508766174316,
  7.587329864501953,
  7.27519416809082,
  7.06112003326416,
  7.101964950561523,
  6.761170387268066,
  6.627472877502441],
 'dev_score_history': [0.6515,
  0.7375,
  0.7558,
  0.7976,
  0.8087,
  0.8054,
  0.8461,
  0.8395,
  0.8514,
  0.8675,
  0.8672,
  0.8741,
  0.8758,
  0.8825,
  0.8847,
  0.8941,
  0.8958,
  0.891,
  0.896,
  0.9019],
 'test_score': 0.8975,
 'train_loss_history': [36.58232749656395,
  22.29057041450783,
  18.298787449024342,
  15.911245897081162,
  14.340392522458677,
  13.20973712073432,
  12.204277582521792,
  11.540283704687047,
  10.911153549618192,
  10.226555640609176,
  9.679706382751466,
  9.236242905369512,
  8.886996791980884,
  8.46040549

# ASSIGNMENT 2

In this assignment we will be using the trained model in the previous step to tag some texts.

**NOTE**: If you use the Basque corpus, you can get a document to process in Basque from a newspaper: https://www.berria.eus/

+ TODO: Pick a document of your choice and run the following Flair components:
  + Tokenize and segment the document into sentences.
  + Instiantiate a SequenceTagger with the generated lemmatizer model.
  + Lemmatize the sentences and print the results (ideally saving the predictions into a list of Sentence objects).

In [ ]:
# TODO add your code here

# ASSIGNMENT 3

As you can see in the previous step, the model we trained predicts this weird lemma_rules based on the minimum script edits generated by the get_ses_affixes.py script (in the resources folder). In order to obtain the real lemma we need to decode the lemma_rule using the function _apply_lemma_rule(form, lemma_rule) below.

+ This function takes as parameters the original word we want to lemmatize and the lemma_rule predicted by our lemmatizer model.

  ```
  decoded_lemma = _apply_lemma_rule('partidua', '↓0;d¦-+')
  print(decoded_lemma)
  'partidu'
  ```
Ideally you may have saved the previous predictions in a list of Sentence objects. Being that the case, you need to:

+ TODO: Iterate over the sentences to extract the lemma prediction (or lemma_rule) which will be used, together with the word, as input to obtain the decoded lemma.
+ TODO: print the results with the decoded lemma.
 

In [ ]:
def _apply_lemma_rule(form, lemma_rule):
    if ';' not in lemma_rule:
        raise ValueError('lemma_rule %r for form %r missing semicolon' % (lemma_rule, form))
    casing, rule = lemma_rule.split(";", 1)
    if rule.startswith("a"):
        lemma = rule[1:]
    else:
        form = form.lower()
        rules, rule_sources = rule[1:].split("¦"), []
        assert len(rules) == 2
        for rule in rules:
            source, i = 0, 0
            while i < len(rule):
                if rule[i] == "→" or rule[i] == "-":
                    source += 1
                else:
                    assert rule[i] == "+"
                    i += 1
                i += 1
            rule_sources.append(source)

        try:
            lemma, form_offset = "", 0
            for i in range(2):
                j, offset = 0, (0 if i == 0 else len(form) - rule_sources[1])
                while j < len(rules[i]):
                    if rules[i][j] == "→":
                        lemma += form[offset]
                        offset += 1
                    elif rules[i][j] == "-":
                        offset += 1
                    else:
                        assert (rules[i][j] == "+")
                        lemma += rules[i][j + 1]
                        j += 1
                    j += 1
                    # print(lemma)
                if i == 0:
                    lemma += form[rule_sources[0]: len(form) - rule_sources[1]]
        except:
            lemma = lemma

    for rule in casing.split("¦"):
        if rule == "↓0": continue  # The lemma is lowercased initially
        if not rule: continue  # Empty lemma might generate empty casing rule
        case, offset = rule[0], int(rule[1:])
        lemma = lemma[:offset] + (lemma[offset:].upper() if case == "↑" else lemma[offset:].lower())
    return lemma


In [ ]:
pip install syntok

  Created wheel for syntok: filename=syntok-1.3.1-cp36-none-any.whl size=20918 sha256=295044d996d6782a8e766e603ffc704895f336590e56bb7024d385067a364d42
  Stored in directory: /root/.cache/pip/wheels/51/c6/a4/be1920586c49469846bcd2888200bdecfe109ec421dab9be2d
Successfully built syntok


In [ ]:
# TODO add your code here
from flair.data import Sentence
from flair.models import SequenceTagger
from syntok.tokenizer import Tokenizer
import syntok.segmenter as segmenter


2021-02-17 10:56:34,031 loading file /content/drive/My Drive/resources/eu-lemma-model/best-model.pt
Jeroen	Jeroen
Dijsselbloem	Dijsselbloem
Eurotaldeko	Eurotalde
presidentearekin	presidente
egin	egin
du	*edun
bilera	bilera
Varufakisek	Varufakis
,	,
eta	eta
haren	hura
aurrean	aurre
nabarmendu	nabarmendu
du	*edun
Greziako	Grezia
Gobernua	gobernu
"	"
egungo	egun
erreskate	erreskate
plana	plan
onartzen	onartu
ez	ez
duen	*edun
programa	programa
batengatik	batenga
"	"
aukeratu	aukeratu
zutela	*tedun
herritarrek	herritar
.	.

Hala	hala
ere	ere
,	,
azaldu	azaldu
du	*edun
gobernuaren	gobernu
asmoa	asmo
da	izan
Europako	Europa
kidekin	kideki
elkarlanean	elkarlan
aritzea	aritu
dela	izan
.	.

"	"
Saiatuko	saiatu
gara	izan
gure	gu
kideak	kide
konbentzitzen	konbentzitu
irtenbide	irtenbide
bat	bat
topatu	topatu
behar	behar
dugula	ukan
Europako	Europa
interes	interes
komunen	komun
alde	alde
"	"
,	,
esan	esan
du	*edun
Varufakisek	Varufakis
.	.

Dijsselbloemek	Dijsselbloem
,	,
berriz	berriz
,	,
gogorara